# ReAct agent

1. create_react_agent: (one that creates the agent)

Takes each tool name and description
Formats them into a standardized way the LLM can understand
Inserts them into a specific placeholders in the react prompt template
it makes the LLM call + takes the LLM response and parses the response to one of this two classes `AgentAction`, `AgentFinish`.

`AgentAction`: This is a LangChain class that represents an action the agent wants to take. It typically contains:
 ```
   action = AgentAction(
    tool='calculator'
    tool_input='2+3'
    log= 'I need you to calculate the sum of 2 + 3'
   )
 ```
 this consists of the thought, action and observation. 

 `AgentFinish`: Incase the LLM doesnt have to call any tool this is the final call and contains the final answer.

 ```
  finish = AgentFinish(
    return_values={"output": "The ans is 5"}
    log= "I've calculate the sum of 2+3 and the answer is 5"
  )
 ```

 2. AgentExecutor: 

 It takes the agent from create_react_agent and manages the execution loop.
 Recieves the user's question and feeds it to the agent
 Identifies which tool to run based on the agents output(AgentAction or No tool if AgentFinish)
 Executes the tool and captures the result
 Feeds the result back to the agent for the next decision
 continues this loop until AgentFinish
 Returns the final answer to the user


 # Key advantages of using LangGraph

 LangGraph turns the hidden 'black-box' loop into a visible, editable workflow

 Start-> reason_node --->act_node 
            |              <-|
            v 
           End

The reason_node now does what create_react_agent did. It thinks and decides.
if the reason_node outputs an agentAction then act node executes the tool, The
results flow back to reason node for the next decision.
When the agent has the final answer, It takes the right path to end
This makes the black box of the agentExecutor transparent and modifiable

In [13]:
# reason node

from langchain_openai import ChatOpenAI
from langchain.agents import tool, create_react_agent
import datetime
from langchain_community.tools import TavilySearchResults
from langchain import hub


llm = ChatOpenAI(model='gpt-4o')

@tool
def get_current_system_time(format: str = "%Y-%m-%d %H:%M:%S") -> str:
    """ Returns the current date and time in the specified format """
    import datetime
    # Get the current system time
    current_time = datetime.datetime.now()
    # Format the time as a string
    formatted_time = current_time.strftime(format)
    return formatted_time

search_tool = TavilySearchResults(search_depth="basic")
tools = [search_tool, get_current_system_time]
react_prompt = hub.pull("hwchase17/react")
react_agent_runnable = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=react_prompt
)



In [14]:
# react state
from typing import TypedDict, Annotated, Union
from langchain_core.agents import AgentAction, AgentFinish
import operator

class AgentState(TypedDict):
    input: str
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]


In [5]:
from dotenv import load_dotenv
load_dotenv()

def reason_node(state: AgentState):
    agent_outcome = react_agent_runnable.invoke(state)
    return{"agent_outcome": agent_outcome}

def act_node(state: AgentState):
    agent_action = state["agent_outcome"]
    # extract tool name and input from AgentAction
    tool_name = agent_action.tool
    tool_input = agent_action.tool_input
    tool_function = None
    for tool in tools:
        if tool.name == tool_name:
            tool_function = tool
            break
    
    if tool_function:
        if isinstance(tool_input, dict):
            output = tool_function.invoke(**tool_input)
        else:
            output = tool_function.invoke(tool_input)
    else:
        output = f"Tool '{tool_name}' not found"
    return{"intermediate_steps":[(agent_action, str(output))]}



In [15]:
# graph
from dotenv import load_dotenv
from langchain_core.agents import AgentFinish, AgentAction
from langgraph.graph import StateGraph, END

load_dotenv()

REASON_NODE = "reason_node"
ACT_NODE = "act_node"

def should_continue(state: AgentState):
    if isinstance(state["agent_outcome"], AgentFinish):
        import pdb
        pdb.set_trace()
        return END
    else:
      return ACT_NODE


graph = StateGraph(state_schema=AgentState)

graph.add_node(REASON_NODE, reason_node)
graph.add_node(ACT_NODE, act_node)
graph.set_entry_point(REASON_NODE)

graph.add_conditional_edges(
    REASON_NODE,
    should_continue
)

graph.add_edge(REASON_NODE, ACT_NODE)
app = graph.compile()
result = app.invoke(
    {
        "input": "How many days ago was the latest SpaceX launch?", 
        "agent_outcome": None, 
        "intermediate_steps": []
    }
)

print(result, 'result---')
# print(result["agent_outcome"].return_values["output"], "final result")

agent_outcome = result["agent_outcome"]

if hasattr(agent_outcome, "return_values"):
    print(agent_outcome.return_values["output"], "final result")
else:
    print("Agent is still reasoning, or returned an intermediate action:", agent_outcome)


{'input': 'How many days ago was the latest SpaceX launch?', 'agent_outcome': AgentAction(tool='get_current_system_time', tool_input="'%Y-%m-%d'", log="To answer the question of how many days ago the latest SpaceX launch occurred, I need to find the date of the most recent SpaceX launch and then compare it to today's date. \n\nAction: get_current_system_time\nAction Input: '%Y-%m-%d'"), 'intermediate_steps': [(AgentAction(tool='get_current_system_time', tool_input="'%Y-%m-%d'", log="To answer the question of how many days ago the latest SpaceX launch occurred, I need to find the date of the most recent SpaceX launch and then compare it to today's date. \n\nAction: get_current_system_time\nAction Input: '%Y-%m-%d'"), "'2025-06-20'")]} result---
Agent is still reasoning, or returned an intermediate action: tool='get_current_system_time' tool_input="'%Y-%m-%d'" log="To answer the question of how many days ago the latest SpaceX launch occurred, I need to find the date of the most recent Sp